---

Created for [learn-investments.rice-business.org](https://learn-investments.rice-business.org)
    
By [Kerry Back](https://kerryback.com) and [Kevin Crotty](https://kevincrotty.rice.edu/)
    
Jones Graduate School of Business, Rice University

---


# EXAMPLE DATA

In [8]:
import numpy as np

R = 30*12                      # 30 years until retirement
T = 60*12                      # 60 total years
B0 = 100000                    # initial balance is $100,000
D1 = 1000                      # initial savings is $1,000 (per month)
W1 = 10000                     # withdraw $10,000 first month in retirement
g = 0.002                      # deposit is 0.2% larger each month
h = 0                          # withdrawals are constant
mn = 0.06 / 12                 # earn 1/2% per month on average
sd = 0.30 / np.sqrt(12)        # risk annualizes to 30%
numsims = 5000                 # 5,000 simulations

# CALCULATE CASH FLOWS

In [9]:
D = D1 * (1+g)**np.arange(R)
W = W1 * (1+h)**np.arange(T-R)
B0 = np.concatenate(([B0], np.zeros(T)))
D = np.concatenate(([0], D, np.zeros(T-R)))
W = np.concatenate((np.zeros(R), W, [0]))
CF = B0 + D - W

# SIMULATE FUTURE VALUE FACTORS

Future value of a cash flow $x$ at date $t$ is $$x(1+r_{t+1})(1+r_{t+2})\cdots (1+r_T).$$ fvFactors contains the products of the (1+r) factors for each date $t=0, \ldots, T-1$ and each simulation.

In [10]:
import pandas as pd

rets = np.random.normal(loc=mn, scale=sd, size=(T, numsims))
rets = pd.DataFrame(rets)

def fvs(rets):
    x = np.flip(np.cumprod(1+rets))
    return np.concatenate((x, [1]))

fvFactors = rets.apply(fvs)

# CALCULATE ENDING BALANCES

In [11]:
BT = fvFactors.multiply(CF, axis=0).sum()
BT.describe(percentiles=(0.1, 0.25, 0.5, 0.75, 0.9))

count    5.000000e+03
mean     5.834395e+05
std      5.941405e+07
min     -1.760484e+08
10%     -1.107132e+07
25%     -5.345616e+06
50%     -2.400029e+06
75%     -8.889813e+05
90%      4.495223e+06
max      3.014614e+09
dtype: float64

# FIGURE 1

In [12]:
import plotly.graph_objects as go

trace = go.Box(x=BT, hovertemplate="%{x}", name="")
fig = go.Figure(trace)
fig.update_layout(
    yaxis_title="",
    xaxis_title="Ending Balance",
    template="plotly_white",
)
fig.show()

# FIGURE 2

In [13]:
grid = [i / 100 for i in range(1, 100)]
pcts = BT.quantile(grid).to_numpy()
trace = go.Scatter(
    x=grid,
    y=pcts,
    mode="lines",
    hovertemplate="%{x:.0%} percentile=%{y:,.1f}MM<extra></extra>"
)
fig = go.Figure(trace)
fig.update_layout(
    xaxis_title="Percentile",
    yaxis_title="Ending Balance",
    xaxis_tickformat=".0%",
    template="plotly_white",
)
fig.show()